In [1]:
# !pip install tensorflow
# !pip install pandas
# !pip install scikit-learn
# !pip install transformers
# !pip install torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.7 MB/s eta 0:00:00


In [ ]:
# !gdown --folder https://drive.google.com/drive/folders/1CAbb3DjrOPBNm0ozVBfhvrEh9P9rAppc

Retrieving folder list
Processing file 1e_G-9a66AryHxBOwGWhriePYCCa4_29e subtaskA_dev_monolingual.jsonl
Processing file 123UQ92LxtHaVTbNYlmjnG1CWwD-x7wDL subtaskA_dev_multilingual.jsonl
Processing file 1HeCgnLuDoUHhP-2OsTSSC3FXRLVoI6OG subtaskA_train_monolingual.jsonl
Processing file 13-9-DakCeLFbPgCiVIU0v6_BCQx0ppz6 subtaskA_train_multilingual.jsonl
Retrieving folder list completed
Building directory structure
Building directory structure completed


In [1]:
import json
import gc
import torch
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

C:\Users\zuba1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the pre-trained BERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

# Set device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

C:\Users\zuba1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\zuba1\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of DistilBertForSequenceClassification w

In [3]:
# Load the data from the JSONL file into a pandas DataFrame
with open('/data_json/SubtaskA/subtaskA_train_monolingual.jsonl', 'r') as f:
    df = pd.read_json(f, lines=True, orient='records')

FileNotFoundError: [Errno 2] No such file or directory: '/data_json/SubtaskA/subtaskA_train_monolingual.jsonl'

In [ ]:
class task_A_dataloader(Dataset):
    def __init__(self, df, text, label, model, source, id):
        self.text = df['text']
        self.label = df['label']
        self.model = df['model']
        self.source = df['source']
        self.id = df['id']

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        item_label = self.label[idx]
        item_text = self.text[idx]
        item_model = self.model[idx]
        item_source = self.source[idx]
        item_id = self.id[idx]
        return item_text, item_label, item_model, item_source, item_id

In [ ]:
# Set device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Encode the input data using the tokenizer on the GPU
encoded_data = []
with torch.cuda.device(0):
  for text in tqdm(df['text'].values.tolist()):
    encoded_text = tokenizer.encode_plus(text, add_special_tokens=True, return_attention_mask=True, padding='max_length', max_length=256, truncation=True, verbose=True)
    encoded_data.append(encoded_text)

TypeError: ignored

In [ ]:

# Split the data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Create an instance of the task_A_dataloader class for the test set
test_dataset = task_A_dataloader(test_df, 'text', 'label', 'model', 'source', 'id')
train_dataset = task_A_dataloader(train_df, 'text', 'label', 'model', 'source', 'id')

In [ ]:
# Encode the input data using the tokenizer on the GPU
encoded_data_train = []
with torch.cuda.device(0):
  for text in tqdm(train_dataset.text.tolist()):
    encoded_text = tokenizer.encode_plus(text, add_special_tokens=True, return_attention_mask=True, padding='max_length', max_length=256, truncation=True, verbose=True)
    encoded_data_train.append(encoded_text)

In [ ]:
# Encode the input data using the tokenizer on the GPU
encoded_data_test = []
with torch.cuda.device(0):
  for text in tqdm(test_dataset.text.tolist()):
    encoded_text = tokenizer.encode_plus(text, add_special_tokens=True, return_attention_mask=True, padding='max_length', max_length=256, truncation=True, verbose=True)
    encoded_data_test.append(encoded_text)

In [ ]:
train_input_ids = np.array([data['input_ids'] for data in encoded_data_train])
train_attention_masks = np.array([data['attention_mask'] for data in encoded_data_train])
train_labels = np.array(train_df['label'])

val_input_ids = np.array([data['input_ids'] for data in encoded_data_test])
val_attention_masks = np.array([data['attention_mask'] for data in encoded_data_test])
val_labels = np.array(test_df['label'])

In [ ]:
# Define the optimizer, loss, and metric
optimizer = torch.optim.Adam(params=model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()
metric = accuracy_score

# Define the training loop
def train(model, loss_fn, metric, optimizer, train_loader, val_loader, epochs=10, device):
    # Move the model to the specified device
    model.to(device)

    # Initialize lists to store the training loss and accuracy for each epoch
    train_loss_list = []
    train_acc_list = []
    val_loss_list = []
    val_acc_list = []

    # Training loop
    for epoch in range(epochs):
        # Set the model to training mode
        model.train()

        # Initialize variables to keep track of the total loss and number of correct predictions
        total_loss = 0.0
        total_correct = 0

        # Iterate over the training data
        for inputs, attention_mask, labels in train_loader:
            # Move the inputs and labels to the specified device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs, attention_mask=attention_mask)

            # Extract the logits from the SequenceClassifierOutput object
            logits = outputs.logits

            # Compute the loss
            loss = loss_fn(logits, labels)

            # Backward pass
            loss.backward()

            # Update the weights
            optimizer.step()

            # Update the total loss
            total_loss += loss.item()

            # Compute the predicted labels
            _, predicted = torch.max(outputs, 1)

            # Update the total number of correct predictions
            total_correct += (predicted == labels).sum().item()

        # Compute the average training loss and accuracy for the epoch
        train_loss = total_loss / len(train_loader)
        train_acc = total_correct / len(train_loader.dataset)

        # Append the training loss and accuracy to the lists
        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)

        # Set the model to evaluation mode
        model.eval()

        # Initialize variables to keep track of the total loss and number of correct predictions for validation
        val_loss = 0.0
        val_correct = 0

        # Disable gradient calculation
        with torch.no_grad():
            # Iterate over the validation data
            for inputs, attention_mask, labels in val_loader:
                # Move the inputs and labels to the specified device
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(inputs, attention_mask=attention_mask)
                
                # Extract the logits from the SequenceClassifierOutput object
                logits = outputs.logits

                # Compute the loss
                loss = loss_fn(logits, labels)

                # Update the total loss
                val_loss += loss.item()

                # Compute the predicted labels
                _, predicted = torch.max(outputs, 1)

                # Update the total number of correct predictions
                val_correct += (predicted == labels).sum().item()

        # Compute the average validation loss and accuracy for the epoch
        val_loss = val_loss / len(val_loader)
        val_acc = val_correct / len(val_loader.dataset)

        # Append the validation loss and accuracy to the lists
        val_loss_list.append(val_loss)
        val_acc_list.append(val_acc)

        # Print the training and validation metrics for the epoch
        print(f"Epoch {epoch+1}/{epochs}:")
        print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.4f}")
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")
        print()

    # Return the training and validation metrics
    return train_loss_list, train_acc_list, val_loss_list, val_acc_list

# Train the model using the training loop
train_loss_list, train_acc_list, val_loss_list, val_acc_list = train(model, loss_fn, metric, optimizer, train_tensors, val_tensors)

# Print the training and validation metrics
print("Training Loss:", train_loss_list[-1])


In [ ]:
# Iterate over the training data
for inputs, attention_mask, labels in train_loader:
    # Move the inputs, attention_mask, and labels to the specified device
    inputs = inputs.to(device)
    attention_mask = attention_mask.to(device)
    labels = labels.to(device)

    # Rest of your code...

# Disable gradient calculation
with torch.no_grad():
    # Iterate over the validation data
    for inputs, attention_mask, labels in val_loader:
        # Move the inputs, attention_mask, and labels to the specified device
        inputs = inputs.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # Rest of your code...

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

# Run the garbage collector
gc.collect()

# Clean the GPU cache
torch.cuda.empty_cache()

In [ ]:
# Train the model using the training loop
train_loss_list, train_acc_list, val_loss_list, val_acc_list = train(model, loss_fn, metric, optimizer, train_loader, val_loader, epochs, device)

# Extract the logits from the SequenceClassifierOutput object
train_logits = train_loss_list.logits
val_logits = val_loss_list.logits

# Convert the logits to tensors
train_logits_tensor = torch.tensor(train_logits)
val_logits_tensor = torch.tensor(val_logits)

# Print the training and validation metrics
print("Training Loss:", train_loss_list[-1])
print("Training Accuracy:", train_acc_list[-1])
print("Validation Loss:", val_loss_list[-1])
print("Validation Accuracy:", val_acc_list[-1])

In [ ]:
def train(model, loss_fn, metric, optimizer, train_loader, val_loader, epochs, device):
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            # Move inputs and labels to the correct device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Extract the logits from the SequenceClassifierOutput object
            logits = outputs.logits

            # Compute the loss
            loss = loss_fn(logits, labels)

            # Rest of your code...

In [ ]:
# Iterate over the training data
for inputs, attention_mask, labels in train_loader:
    # Move the inputs, attention_mask, and labels to the specified device
    inputs = inputs.to(device)
    attention_mask = attention_mask.to(device)
    labels = labels.to(device)

    # Rest of your code...

# Disable gradient calculation
with torch.no_grad():
    # Iterate over the validation data
    for inputs, attention_mask, labels in val_loader:
        # Move the inputs, attention_mask, and labels to the specified device
        inputs = inputs.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # Rest of your code...